In [3]:
import attr
import xsimlab as xs


In [2]:
def create_process(dim):
    
    @xs.process
    class A:
        x = xs.index(dims=dim)
        
        def initialize(self):
            self.x = [0, 1, 2, 3]
    
    return A

In [3]:
B = create_process('y')

In [4]:
B

__main__.create_process.<locals>.A

In [5]:
C = create_process('z')

In [6]:
C

__main__.create_process.<locals>.A

In [7]:
xs.process_info(B)

<A  (xsimlab process)>
Variables:
    x          [out] ('y',) 
Simulation stages:
    initialize


In [8]:
xs.process_info(C)

<A  (xsimlab process)>
Variables:
    x          [out] ('z',) 
Simulation stages:
    initialize


In [2]:
@xs.process
class X:
    pass

In [196]:
# do not decorate A with attr.s or xs.process!!

class A(X):
    index = xs.index(dims=('not_initialized'))
    label = xs.variable(intent='in')
    dim = xs.variable(intent='in')
    
    output = xs.variable(intent='out')

    def initialize(self):
        self.index = [f"{self.label}-{i}" for i in range(self.dim)]
        
        print(self.index)
        print(self.racke)

In [34]:
def make_process(cls_name, label):
    new_cls = type(cls_name, A.__bases__, dict(A.__dict__))
    
    new_cls.index.metadata['dims'] = label
    
    new_cls.output.metadata['dims'] = ((label, 'time'),)
    
    print(new_cls.index.metadata)
    new_cls.index.init = False
    #new_cls.dim.init = False
    
    racke = attr.ib()
    
    #print(new_cls.racke)
    
    return xs.process(new_cls)


In [35]:
a = (1,)

name = 'Hello'

a = 'name'

a

'name'

In [36]:
# what am I trying to do here

1. So each "forcing flux" is defined for a single component
2. but I want the functionality, that these can be easily shared between components

SyntaxError: invalid syntax (<ipython-input-36-1e789711c185>, line 3)

In [18]:
def make_forcing(cls_in, label):
    new_cls = type(label, cls_in.__bases__, dict(cls_in.__dict__))
    
    new_cls.labels.metadata['dims'] = label
    
    print(new_cls.labels)
    print(new_cls.labels.metadata)

    return xs.process(new_cls)


In [49]:
@xs.process
class Base:
    GetMyStuff = xs.variable()

class InputMultiComps(Base):
    
    labels = xs.variable(intent='in', dims='comps')
    
    
    def initialize(self):
        print('dict', self.__dict__['__xsimlab_name__'], 'dict')
        print(self.labels)
        print(self.GetMyStuff)
        
    @xs.run_step(level==0, args=None)
    def run_step_1(self):
        print('heeeeello')

AttributeError: module 'xsimlab' has no attribute 'run_step'

In [42]:
InputMultiComps.__bases__

(__main__.Base,)

In [43]:
import numpy as np

a = ['X']

a_it = np.nditer(a)#, flags=['zerosize_ok', 'refs_ok', 'multi_index'])

for label in a:
    print(label, type(label))
    
while not a_it.finished:
    print(str(a_it.value), type(a_it.value))
    a_it.iternext()

X <class 'str'>
X <class 'numpy.ndarray'>


In [44]:
test = make_forcing(InputMultiComps,'Trest')

_CountingAttr(counter=45, _default=NOTHING, repr=True, eq=True, order=True, hash=None, init=True, metadata={'var_type': <VarType.VARIABLE: 'variable'>, 'dims': 'Trest', 'intent': <VarIntent.IN: 'in'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}})
{'var_type': <VarType.VARIABLE: 'variable'>, 'dims': 'Trest', 'intent': <VarIntent.IN: 'in'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}}


In [45]:
test

__main__.Trest

In [46]:
mod = xs.Model({'ForcingFlux':test})

In [47]:
in_ds = xs.create_setup(model=mod,
               clocks={'clock': [0, 1]},
               input_vars={
                   'ForcingFlux__labels':['hello','no','dont'],
                   
                   'ForcingFlux__GetMyStuff':1
               },
                output_vars={})

In [48]:
with mod:
    out_ds = in_ds.xsimlab.run()

dict ForcingFlux dict
['hello' 'no' 'dont']
1


In [61]:
B = make_process('B', 'y')

NameError: name 'A' is not defined

In [251]:
B

__main__.B

In [252]:
xs.process_info(B)

<B  (xsimlab process)>
Variables:
    index      [out] y 
    label       [in]
    dim         [in]
    output     [out] ('y', 'time') 
Simulation stages:
    initialize


In [253]:
model = xs.Model({'Bx':B})

In [254]:
in_ds = xs.create_setup(model=model,
                clocks={'clock': [0, 1]},
               input_vars={
                   'Bx__dim':20,
                   'Bx__label':'Phytoplankton'
               },
                output_vars={}
               )

In [255]:
with model:
    out_ds = in_ds.xsimlab.run()

['Phytoplankton-0', 'Phytoplankton-1', 'Phytoplankton-2', 'Phytoplankton-3', 'Phytoplankton-4', 'Phytoplankton-5', 'Phytoplankton-6', 'Phytoplankton-7', 'Phytoplankton-8', 'Phytoplankton-9', 'Phytoplankton-10', 'Phytoplankton-11', 'Phytoplankton-12', 'Phytoplankton-13', 'Phytoplankton-14', 'Phytoplankton-15', 'Phytoplankton-16', 'Phytoplankton-17', 'Phytoplankton-18', 'Phytoplankton-19']


AttributeError: 'B' object has no attribute 'racke'

In [20]:
out_ds

<xarray.Dataset>
Dimensions:    (clock: 2, y: 20)
Coordinates:
  * clock      (clock) int64 0 1
Dimensions without coordinates: y
Data variables:
    Bx__dim    int64 20
    Bx__label  <U13 'Phytoplankton'
    index      (y) <U16 'Phytoplankton-0' ... 'Phytoplankton-19'

In [64]:
#C = make_process('C', 'z')

In [100]:
new_cls = type('Hey', A.__bases__, dict(A.__dict__))

In [102]:
new_cls.index.metadata['dims'] = (('q', 'P'),)

new_cls.index.metadata['dims']

(('q', 'P'),)

In [103]:
new_cls.dim.metadata

{'var_type': <VarType.VARIABLE: 'variable'>,
 'dims': ((),),
 'intent': <VarIntent.OUT: 'out'>,
 'groups': (),
 'static': False,
 'attrs': {},
 'description': '',
 'encoding': {},
 'default': 3}

In [67]:

new_cls.q.metadata['default']

KeyError: 'default'

In [32]:
xs.process_info(C)

<C  (xsimlab process)>
Variables:
    x          [out] z 
    q          [out] z or time 
Simulation stages:
    initialize


In [132]:
@xs.process
class ForcingFlux:
    """"""
    flux = xs.any_object()
    
    flux_func = xs.on_demand()
    
    def initialize(self):
        """ basic initialisation of forcing """
        self.flux = self.flux_func

    @flux_func.compute
    def fluxfunction(self):
        """ returns a function that is used in environment to calculate flux """
        
        def hello_print(arg='No Hello'):
            print(arg)
        return hello_print #ValueError('interpolate function needs to be initialized in subclass of forcing')


@xs.process
class Base:
    hello = xs.on_demand()
    
    def initialize(self):
        self.world = self.hello()
    
    @hello.compute
    def interpolate(self):
        """ returns interpolated scipy object, unit : {d^-1} """
        raise ValueError('interpolate function needs to be initialized in subclass of forcing')


@xs.process
class Add2Base(Base):
    
    hello = xs.on_demand()
    
    @hello.compute
    def initialize(self):
        return 'hello'
        
@xs.process
class Environment:
    """THIS IS A COLLECTION OF ALL COMPONENTS"""
    hello_in = xs.foreign(Base, 'hello')
    
    hello_flux = xs.foreign(ForcingFlux, 'flux')
    
    extra_args = xs.group('forcing_pars')
    
    context_args = xs.any_object()
    
    
    def initialize(self):
        print(self.hello_in)
        comp_self.args = self.extra_args
        self.hello_flux(args='None')


#PAR'S CONTEXTDICT!
## need to use a defaultdict, and check for presence of par in dict..
## but the question is where I can provide the component specific pars?

### can I add that to make_Component() somehow? that each env adds that automatically?

### xs.variable(intent='inout') <- parameter in environment
### added as xs.foreign(intent='out') <- parameter as added to component via function

### xs.foreign()


In [135]:
@xs.process
class IndexTEST:
    test = xs.index(dims='test')
    
    def initialize(self):
        self.test = np.arange(5)
        print(self.test)

In [137]:
IndexTEST().initialize()

[0 1 2 3 4]


In [133]:
model = xs.Model({'FX':ForcingFlux, 'B':Add2Base, 'X':Environment})

In [134]:
in_ds = xs.create_setup(model=model,
               clocks={'time':[0,1]},
            input_vars={'X__args':{'P':1}})

KeyError: "('X__args', 'P') is/are not valid key(s) for input variables in model <xsimlab.Model (3 processes, 0 inputs)>\nFX\nB\nX\n"

In [127]:
with model:
    in_ds.xsimlab.run()

ValueError: unable to infer dtype on variable None; xarray cannot serialize arbitrary Python objects

In [36]:
import numpy as np

np.full(3, 1)

array([1, 1, 1])

In [29]:
from xsimlab import RuntimeHook, runtime_hook

import time

In [30]:
class PrintStepTime(RuntimeHook):

    @runtime_hook('run_step', 'model', 'pre')
    def start_step(self, model, context, state):
        self._start_time = time.time()
        print(f"Starting step {context['step']}")

    @runtime_hook('run_step', 'model', 'post')
    def finish_step(self, model, context, state):
        step_time = time.time() - self._start_time
        print(f"Step {context['step']} took {step_time} seconds")


In [31]:
with PrintStepTime():
    in_ds.xsimlab.run(model=model)

['Phytoplankton-0', 'Phytoplankton-1', 'Phytoplankton-2', 'Phytoplankton-3', 'Phytoplankton-4', 'Phytoplankton-5', 'Phytoplankton-6', 'Phytoplankton-7', 'Phytoplankton-8', 'Phytoplankton-9', 'Phytoplankton-10', 'Phytoplankton-11', 'Phytoplankton-12', 'Phytoplankton-13', 'Phytoplankton-14', 'Phytoplankton-15', 'Phytoplankton-16', 'Phytoplankton-17', 'Phytoplankton-18', 'Phytoplankton-19']
Starting step 0
Step 0 took 4.100799560546875e-05 seconds
